In [1]:
import pandas as pd
import numpy as np
import re
import requests #Основная библиотека для отправки http запросов. 
from bs4 import BeautifulSoup
import time

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # библиотека для визуализации статистических данных

# Указать путь размещения файлов main_task.csv, kaggle_task.csv, sample_submission.csv
data_path = 'Data/'

df_train=pd.read_csv(data_path+'main_task.csv')
df_test=pd.read_csv(data_path+'kaggle_task.csv')
sample_submission = pd.read_csv(data_path+'sample_submission.csv')

In [3]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [4]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [5]:
# Часть ссылки из датафрейма
data['URL_TA'][10489]

'/Restaurant_Review-g274887-d2171461-Reviews-Claro_Bistro-Budapest_Central_Hungary.html'

In [6]:
# Создаем лист ссылок
list_of_links = list(map(lambda x: "https://www.tripadvisor.ru" + x, data.loc[:, "URL_TA"].tolist()))
list_of_links[10489]

'https://www.tripadvisor.ru/Restaurant_Review-g274887-d2171461-Reviews-Claro_Bistro-Budapest_Central_Hungary.html'

#### 2.7.1 Photo count_value on site

In [7]:
import requests #Основная библиотека для отправки http запросов. 
from bs4 import BeautifulSoup
r = requests.get('https://www.tripadvisor.ru' + data['URL_TA'][10489])
soup = BeautifulSoup(r.text)
count_photo_text = soup.select_one('.details').text

pattern_numb = re.compile('\d+')
pattern_numb.findall(count_photo_text)[0]

'60'

In [37]:
# Парсим страницы с Tripadvisor - смотрим количество фоток на странице
# требует доработки - выбор большего количества данных
pattern_numb = re.compile('\d+')
def numb_of_photos(index_link, link):
    r = requests.get(link)
    soup = BeautifulSoup(r.text)
    if str(soup.select_one('.details')) == 'None':
        file_numb = open("/home/ostrebko/RealDS/Module_1/Data/numb_of_photos.csv", "a")
        file_numb.write(str(index_link) + ',' + '0' + ',')
        file_numb.write(link + "\n")
        file_numb.close()
        return 0
    else:
        file_numb = open("/home/ostrebko/RealDS/Module_1/Data/numb_of_photos.csv", "a")
        file_numb.write(str(index_link) + ',' + pattern_numb.findall(soup.select_one('.details').text)[0] + ',')
        file_numb.write(link + "\n")
        file_numb.close()
        return int(pattern_numb.findall(soup.select_one('.details').text)[0])

90

In [42]:
list_numb_photos = []
count_for_print = -1
for link in list_of_links[0:50000]:
    count_for_print += 1
    if count_for_print%50 == 0:
        time.sleep(5)
        print(count_for_print)
    list_numb_photos.append(numb_of_photos(list_of_links.index(link), link))

0
2
4
6
8
